In [ ]:
%pip install -U azure-ai-ml

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)
from azure.ai.ml.entities import MarketplaceSubscription, ServerlessEndpoint

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    client = MLClient.from_config(credential=credential)
except:
    print("Please create a workspace configuration file in the current directory.")


In [ ]:
list(client.models.list())

In [ ]:
from utils import wait_for_model
model = wait_for_model(client, "ft-raft-wakeboarding-DEMO")
model

In [ ]:
# Do this if needed, does not work currently
# but llama3 example works from here - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-serverless?tabs=python

#model_id="azureml://registries/azureml-meta/models/Llama-2-7b"
base_model_id = model.properties['baseModelId']
model_id = model.id
subscription_name=base_model_id.split('/')[-1]
(subscription_name, model_id)

In [ ]:
model

In [ ]:
model_id = f"azureml://locations/westus3/{"/".join(model.id.split('/')[7:])}"
model_id

In [ ]:
marketplace_subscription = MarketplaceSubscription(
    model_id=base_model_id,
    name=subscription_name,
)

marketplace_subscription = client.marketplace_subscriptions.begin_create_or_update(
    marketplace_subscription
).result()


In [ ]:
# Deploy the model as a serverless endpoint
import uuid
guid = uuid.uuid4()
short_guid = str(guid)[:4]
endpoint_name=f"{model.name}-{short_guid}" # Name must be unique
print(f"Deploying model {model.name} as endpoint {endpoint_name}")

In [ ]:
model_id = "azureml://locations/westus3/workspaces/24827e2c-b602-428c-943b-e9c0204b82cf/models/ft-raft-wakeboarding-DEMO/versions/1"

In [ ]:
serverless_endpoint = ServerlessEndpoint(
    name=endpoint_name,
    model_id=model_id
)

created_endpoint = client.serverless_endpoints.begin_create_or_update(
    serverless_endpoint
).result()

In [ ]:
endpoint = client.serverless_endpoints.get(endpoint_name)
endpoint

In [ ]:
endpoint_keys = client.serverless_endpoints.get_keys(endpoint_name)

In [ ]:
# Update the shared `.env.state` env file with the newly deployed finetuned model endpoint
from utils import update_state
update_state("FINETUNED_OPENAI_BASE_URL", endpoint.scoring_uri)
update_state("FINETUNED_OPENAI_API_KEY", endpoint_keys.primary_key)
update_state("FINETUNED_OPENAI_DEPLOYMENT", endpoint.name)

In [ ]:
import requests

url = f"{endpoint.scoring_uri}/v1/completions"

payload = {
    "max_tokens": 1024,
    "prompt": [
        "Summarize the dialog.\n<dialog>: John: do you play tennis? :-)\r\nIrma: i don't, why do you ask?" +
        "\r\nJohn: you look like someone who plays tennis\r\nIrma: HAHAHAHA i'm literally loling "+
        "\r\nIrma: i don't know what someone who plays tennis looks like\r\nJohn: hahah sorry that was a dumb comment" +
        "\r\nJohn: anyway, would to enroll in a class with me?\r\nJohn: they're dirt cheap at the community center" +
        "\r\nIrma: sounds good, i can wear one of my tennis outfits!!"+
        "\r\nIrma: HAHAHA, i'm still laughing at your comment  lol\n<summary>: "
    ]
}
headers = {
    "Content-Type": "application/json",
    "Authorization": endpoint_keys.primary_key
}

response = requests.post(url, json=payload, headers=headers)

response.json()